This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Interpreting what convnets learn

### Visualizing intermediate activations

In [ ]:
# You can use this to load the file "convnet_from_scratch_with_augmentation.keras" you obtained in the last chapter
from google.colab import files # importing the files module from google colab
files.upload() # uploading the file "convnet_from_scratch_with_augmentation.keras" to google colab

In [ ]:
from tensorflow import keras # importing the keras module from tensorflow
model = keras.models.load_model("convnet_from_scratch_with_augmentation.keras") # loading the model "convnet_from_scratch_with_augmentation.keras"
model.summary() # printing the summary of the model

**Preprocessing a single image**

In [ ]:
from tensorflow import keras # importing the keras module from tensorflow
import numpy as np # importing the numpy module

img_path = keras.utils.get_file( # getting the file "cat.jpg" from the url "https://img-datasets.s3.amazonaws.com/cat.jpg"
    fname="cat.jpg", # setting the file name to "cat.jpg"
    origin="https://img-datasets.s3.amazonaws.com/cat.jpg") # setting the url to "https://img-datasets.s3.amazonaws.com/cat.jpg"

def get_img_array(img_path, target_size): # defining a function "get_img_array" with parameters "img_path" and "target_size"
    img = keras.utils.load_img( # loading the image from the path "img_path"
        img_path, target_size=target_size) # setting the target size to "target_size"
    array = keras.utils.img_to_array(img) # converting the image to an array
    array = np.expand_dims(array, axis=0) # expanding the dimensions of the array
    return array # returning the array

img_tensor = get_img_array(img_path, target_size=(180, 180)) # getting the image array with the target size of (180, 180) (180 pixels by 180 pixels, because the model was trained on images of this size)

**Displaying the test picture**

In [ ]:
import matplotlib.pyplot as plt # importing the matplotlib.pyplot module
plt.axis("off") # turning off the axis
plt.imshow(img_tensor[0].astype("uint8")) # displaying the image
plt.show() # displaying the plot

**Instantiating a model that returns layer activations**

In [ ]:
from tensorflow.keras import layers # importing the layers module from tensorflow.keras

layer_outputs = [] # creating an empty list "layer_outputs"
layer_names = [] # creating an empty list "layer_names"
for layer in model.layers: # iterating through the layers of the model
    if isinstance(layer, (layers.Conv2D, layers.MaxPooling2D)): # checking if the layer is an instance of the Conv2D or MaxPooling2D class
        layer_outputs.append(layer.output) # appending the output of the layer to the list "layer_outputs"
        layer_names.append(layer.name) # appending the name of the layer to the list "layer_names"
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs) # creating a model with the input as the input of the model and the outputs as the list "layer_outputs"

**Using the model to compute layer activations**

In [ ]:
activations = activation_model.predict(img_tensor) # getting the activations of the model on the image tensor

In [ ]:
first_layer_activation = activations[0] # getting the first layer activation
print(first_layer_activation.shape) # printing the shape of the first layer activation

**Visualizing the fifth channel**

In [ ]:
import matplotlib.pyplot as plt # importing the matplotlib.pyplot module
plt.matshow(first_layer_activation[0, :, :, 5], cmap="viridis") # displaying the first layer activation of the 5th filter using the "viridis" colormap (a colormap that goes from yellow for low values to blue for high values)

**Visualizing every channel in every intermediate activation**

In [ ]:
images_per_row = 16 # setting the number of images per row to 16
for layer_name, layer_activation in zip(layer_names, activations): # iterating through the layer names and activations
    n_features = layer_activation.shape[-1] # getting the number of features
    size = layer_activation.shape[1] # getting the size of the layer activation
    n_cols = n_features // images_per_row # getting the number of columns
    display_grid = np.zeros(((size + 1) * n_cols - 1, 
                             images_per_row * (size + 1) - 1)) # creating a display grid with zeros of the size of the layer activation and the number of columns and rows calculated above (with a border of 1 pixel between the images) 
    for col in range(n_cols): # iterating through the columns
        for row in range(images_per_row): # iterating through the rows
            channel_index = col * images_per_row + row # calculating the channel index by multiplying the column by the number of images per row and adding the row index to it 
            channel_image = layer_activation[0, :, :, channel_index].copy() # getting the channel image by copying the channel from the layer activation 
            if channel_image.sum() != 0: # checking if the sum of the channel image is not zero
                channel_image -= channel_image.mean() # subtracting the mean of the channel image from the channel image
                channel_image /= channel_image.std() # dividing the channel image by the standard deviation of the channel image
                channel_image *= 64 # multiplying the channel image by 64
                channel_image += 128 # adding 128 to the channel image
            channel_image = np.clip(channel_image, 0, 255).astype("uint8") # clipping the channel image to be between 0 and 255 and converting it to an unsigned integer
            display_grid[ # setting the display grid to the channel image
                col * (size + 1): (col + 1) * size + col, # setting the column index to the column multiplied by the size plus 1 and the column plus 1 multiplied by the size plus the column
                row * (size + 1) : (row + 1) * size + row] = channel_image # setting the row index to the row multiplied by the size plus 1 and the row plus 1 multiplied by the size plus the row
    scale = 1. / size # setting the scale to 1 divided by the size 
    plt.figure(figsize=(scale * display_grid.shape[1], # setting the figure size to the scale multiplied by the display grid shape
                        scale * display_grid.shape[0])) # setting the figure size to the scale multiplied by the display grid shape 
    plt.title(layer_name) # setting the title of the plot to the layer name
    plt.grid(False) # turning off the grid
    plt.axis("off") # turning off the axis
    plt.imshow(display_grid, aspect="auto", cmap="viridis") # displaying the display grid with the "viridis" colormap and the aspect ratio set to "auto" (so that the images are not stretched) 

### Visualizing convnet filters

**Instantiating the Xception convolutional base**

In [ ]:
model = keras.applications.xception.Xception( # loading the Xception model
    weights="imagenet", # setting the weights to "imagenet" which means that the model is pretrained on the ImageNet dataset
    include_top=False) # setting the include top to False which means that the top layer of the model is not included

**Printing the names of all convolutional layers in Xception**

In [ ]:
for layer in model.layers: # iterating through the layers of the model
    if isinstance(layer, (keras.layers.Conv2D, keras.layers.SeparableConv2D)): # checking if the layer is an instance of the Conv2D or SeparableConv2D class
        print(layer.name) # printing the name of the layer

**Creating a feature extractor model**

In [ ]:
layer_name = "block3_sepconv1" # setting the layer name to "block3_sepconv1"
layer = model.get_layer(name=layer_name) # getting the layer with the name "layer_name"
feature_extractor = keras.Model(inputs=model.input, outputs=layer.output) # creating a model with the input as the input of the model and the output as the output of the layer

**Using the feature extractor**

In [ ]:
activation = feature_extractor( # getting the activation of the feature extractor on the image tensor
    keras.applications.xception.preprocess_input(img_tensor) # preprocessing the image tensor using the preprocess input function of the Xception model
)

In [ ]:
import tensorflow as tf # importing the tensorflow module

def compute_loss(image, filter_index): # defining a function "compute_loss" with parameters "image" and "filter_index"
    activation = feature_extractor(image) # getting the activation of the feature extractor on the image
    filter_activation = activation[:, 2:-2, 2:-2, filter_index] # getting the filter activation of the activation
    return tf.reduce_mean(filter_activation) # returning the mean of the filter activation

**Loss maximization via stochastic gradient ascent**

In [ ]:
@tf.function # compiling the function to a TensorFlow graph
def gradient_ascent_step(image, filter_index, learning_rate): # defining a function "gradient_ascent_step" with parameters "image", "filter_index", and "learning_rate"
    with tf.GradientTape() as tape: # creating a gradient tape
        tape.watch(image) # watching the image
        loss = compute_loss(image, filter_index) # computing the loss of the image and the filter index
    grads = tape.gradient(loss, image) # getting the gradients of the loss with respect to the image
    grads = tf.math.l2_normalize(grads) # normalizing the gradients
    image += learning_rate * grads # adding the learning rate multiplied by the gradients to the image
    return image # returning the image

**Function to generate filter visualizations**

In [ ]:
img_width = 200 # setting the image width to 200
img_height = 200 # setting the image height to 200

def generate_filter_pattern(filter_index): # defining a function "generate_filter_pattern" with the parameter "filter_index"
    iterations = 30 # setting the number of iterations to 30
    learning_rate = 10. # setting the learning rate to 10.
    image = tf.random.uniform( # generating a random uniform image
        minval=0.4, # setting the minimum value to 0.4
        maxval=0.6, # setting the maximum value to 0.6
        shape=(1, img_width, img_height, 3)) # setting the shape to (1, img_width, img_height, 3)
    for i in range(iterations): # iterating through the range of iterations
        image = gradient_ascent_step(image, filter_index, learning_rate) # getting the image after the gradient ascent step
    return image[0].numpy() # returning the image as a numpy array

**Utility function to convert a tensor into a valid image**

In [ ]:
def deprocess_image(image): # defining a function "deprocess_image" with the parameter "image"
    image -= image.mean() # subtracting the mean of the image from the image
    image /= image.std() # dividing the image by the standard deviation of the image
    image *= 64 # multiplying the image by 64
    image += 128 # adding 128 to the image
    image = np.clip(image, 0, 255).astype("uint8") # clipping the image to be between 0 and 255 and converting it to an unsigned integer
    image = image[25:-25, 25:-25, :] # cropping the image
    return image # returning the image

In [ ]:
plt.axis("off") # turning off the axis
plt.imshow(deprocess_image(generate_filter_pattern(filter_index=2))) # displaying the deprocessed image of the generated filter pattern with the filter index of 2

**Generating a grid of all filter response patterns in a layer**

In [ ]:
all_images = [] # creating an empty list "all_images" 
for filter_index in range(64): # iterating through the range of 64
    print(f"Processing filter {filter_index}") # printing the filter index
    image = deprocess_image( # deprocessing the image
        generate_filter_pattern(filter_index) # generating the filter pattern with the filter index
    )
    all_images.append(image) # appending the image to the list "all_images"

margin = 5 # setting the margin to 5
n = 8 # setting the number of images per row to 8
cropped_width = img_width - 25 * 2 # setting the cropped width to the image width minus 25 pixels on each side
cropped_height = img_height - 25 * 2 # setting the cropped height to the image height minus 25 pixels on each side
width = n * cropped_width + (n - 1) * margin # setting the width to the number of images per row multiplied by the cropped width plus the number of images per row minus 1 multiplied by the margin
height = n * cropped_height + (n - 1) * margin # setting the height to the number of images per row multiplied by the cropped height plus the number of images per row minus 1 multiplied by the margin
stitched_filters = np.zeros((width, height, 3)) # creating a matrix of zeros with the width, height, and 3 channels

for i in range(n): # iterating through the range of n
    for j in range(n): # iterating through the range of n
        image = all_images[i * n + j] # getting the image from the list "all_images"
        stitched_filters[ # setting the stitched filters to the image
            (cropped_width + margin) * i : (cropped_width + margin) * i + cropped_width, # setting the cropped width plus the margin multiplied by i to the cropped width plus the margin multiplied by i plus the cropped width
            (cropped_height + margin) * j : (cropped_height + margin) * j + cropped_height, # setting the cropped height plus the margin multiplied by j to the cropped height plus the margin multiplied by j plus the cropped height
            :, # setting the channel to all the channels
        ] = image # setting the stitched filters to the image

keras.utils.save_img( # saving the image
    f"filters_for_layer_{layer_name}.png", stitched_filters) # saving the image as "filters_for_layer_{layer_name}.png"

### Visualizing heatmaps of class activation

**Loading the Xception network with pretrained weights**

In [ ]:
model = keras.applications.xception.Xception(weights="imagenet") # loading the Xception model

**Preprocessing an input image for Xception**

In [ ]:
img_path = keras.utils.get_file( # getting the file "elephant.jpg" from the url "https://img-datasets.s3.amazonaws.com/elephant.jpg"
    fname="elephant.jpg", # setting the file name to "elephant.jpg"
    origin="https://img-datasets.s3.amazonaws.com/elephant.jpg") # setting the url to "https://img-datasets.s3.amazonaws.com/elephant.jpg"

def get_img_array(img_path, target_size): # defining a function "get_img_array" with parameters "img_path" and "target_size"
    img = keras.utils.load_img(img_path, target_size=target_size) # loading the image from the path "img_path"
    array = keras.utils.img_to_array(img) # converting the image to an array
    array = np.expand_dims(array, axis=0) # expanding the dimensions of the array
    array = keras.applications.xception.preprocess_input(array) # preprocessing the input using the preprocess input function of the Xception model
    return array # returning the array

img_array = get_img_array(img_path, target_size=(299, 299)) # getting the image array with the target size of (299, 299) (299 pixels by 299 pixels, because the Xception model was trained on images of this size)

In [ ]:
preds = model.predict(img_array) # getting the predictions of the model on the image array
print(keras.applications.xception.decode_predictions(preds, top=3)[0]) # printing the top 3 predictions of the model

In [ ]:
np.argmax(preds[0]) # getting the index of the maximum prediction

**Setting up a model that returns the last convolutional output**

In [ ]:
last_conv_layer_name = "block14_sepconv2_act" # setting the last convolutional layer name to "block14_sepconv2_act"
classifier_layer_names = [ # creating a list "classifier_layer_names"
    "avg_pool", # setting the first element to "avg_pool"
    "predictions", # setting the second element to "predictions"
]
last_conv_layer = model.get_layer(last_conv_layer_name) # getting the last convolutional layer
last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output) # creating a model with the inputs as the inputs of the model and the output as the output of the last convolutional layer

**Reapplying the classifier on top of the last convolutional output**

In [ ]:
classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:]) # creating an input layer for the classifier
x = classifier_input # setting x to the input layer
for layer_name in classifier_layer_names: # iterating through the classifier layer names
    x = model.get_layer(layer_name)(x) # getting the layer with the name "layer_name" and applying it to x
classifier_model = keras.Model(classifier_input, x) # creating a model with the input as the classifier input and the output as x

**Retrieving the gradients of the top predicted class**

In [ ]:
import tensorflow as tf # importing the tensorflow module

with tf.GradientTape() as tape: # creating a gradient tape
    last_conv_layer_output = last_conv_layer_model(img_array) # getting the output of the last convolutional layer on the image array
    tape.watch(last_conv_layer_output) # watching the last convolutional layer output
    preds = classifier_model(last_conv_layer_output) # getting the predictions of the classifier model on the last convolutional layer output
    top_pred_index = tf.argmax(preds[0]) # getting the index of the maximum prediction
    top_class_channel = preds[:, top_pred_index] # getting the top class channel

grads = tape.gradient(top_class_channel, last_conv_layer_output) # getting the gradients of the top class channel with respect to the last convolutional layer output

**Gradient pooling and channel-importance weighting**

In [ ]:
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy() # getting the pooled gradients by taking the mean of the gradients along the axes (0, 1, 2) and converting it to a numpy array
last_conv_layer_output = last_conv_layer_output.numpy()[0] # getting the last convolutional layer output as a numpy array
for i in range(pooled_grads.shape[-1]): # iterating through the range of the last axis of the pooled gradients
    last_conv_layer_output[:, :, i] *= pooled_grads[i] # multiplying the last convolutional layer output by the pooled gradients
heatmap = np.mean(last_conv_layer_output, axis=-1) # getting the mean of the last convolutional layer output along the last axis

**Heatmap post-processing**

In [ ]:
heatmap = np.maximum(heatmap, 0) # setting the heatmap to the maximum of the heatmap and 0
heatmap /= np.max(heatmap) # dividing the heatmap by the maximum of the heatmap
plt.matshow(heatmap) # displaying the heatmap

**Superimposing the heatmap on the original picture**

In [ ]:
import matplotlib.cm as cm # importing the matplotlib.cm module

img = keras.utils.load_img(img_path) # loading the image from the path "img_path"
img = keras.utils.img_to_array(img) # converting the image to an array

heatmap = np.uint8(255 * heatmap) # converting the heatmap to an unsigned integer

jet = cm.get_cmap("jet") # getting the "jet" colormap
jet_colors = jet(np.arange(256))[:, :3] # getting the jet colors    
jet_heatmap = jet_colors[heatmap] # getting the jet heatmap

jet_heatmap = keras.utils.array_to_img(jet_heatmap) # converting the jet heatmap to an image
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0])) # resizing the jet heatmap to the shape of the image
jet_heatmap = keras.utils.img_to_array(jet_heatmap) # converting the jet heatmap to an array

superimposed_img = jet_heatmap * 0.4 + img # superimposing the jet heatmap on the image
superimposed_img = keras.utils.array_to_img(superimposed_img) # converting the superimposed image to an image

save_path = "elephant_cam.jpg" # setting the save path to "elephant_cam.jpg"
superimposed_img.save(save_path) # saving the superimposed image to the save path

## Summary